# Local Manga AI

This notebook launches a fully-local pipeline: **Qwen2.5 → SDXL → Page Composer** and can optionally expose a public HTTPS URL via **Cloudflare Tunnel**.

Model folders:
- `models/qwen2.5/`
- `models/sdxl/`

In [ ]:
import os
import warnings

# Suppress Hugging Face Hub deprecation warnings
warnings.filterwarnings("ignore", category=UserWarning, message=".*resume_download.*")
warnings.filterwarnings("ignore", category=UserWarning, message=".*local_dir_use_symlinks.*")

# Runtime settings (run this BEFORE loading any torch/diffusers models)
os.environ.setdefault("PYTORCH_CUDA_ALLOC_CONF", "expandable_segments:True")

# Hugging Face token for gated models
os.environ["MANGA_AI_HF_TOKEN"] = "www"

print("PYTORCH_CUDA_ALLOC_CONF=", os.environ.get("PYTORCH_CUDA_ALLOC_CONF"))
print("Hugging Face token set for gated models")
print("HF deprecation warnings suppressed")

In [ ]:
import sys
from pathlib import Path

# Detect project root (folder that contains both ./scripts and ./models)
CWD = Path.cwd().resolve()
ROOT = CWD

if not ((ROOT / "scripts").exists() and (ROOT / "models").exists()):
    if (ROOT.parent / "scripts").exists() and (ROOT.parent / "models").exists():
        ROOT = ROOT.parent.resolve()

if not ((ROOT / "scripts").exists() and (ROOT / "models").exists()):
    raise RuntimeError(
        "Could not locate project ROOT. Expected folders 'scripts' and 'models' in the working directory (or its parent). "
        f"CWD={CWD}"
    )

# Make sure imports like `from scripts...` work
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("CWD:", CWD)
print("ROOT:", ROOT)
print("Python:", sys.version)


In [ ]:
import os
from pathlib import Path

from scripts.model_downloader import ensure_models_downloaded

QWEN_DIR = Path(ROOT) / "models" / "qwen2.5"
QWEN_DIR.mkdir(parents=True, exist_ok=True)
print("Qwen dir:", QWEN_DIR)

try:
    ensure_models_downloaded(
        qwen_dir=QWEN_DIR,
        sdxl_dir=Path(ROOT) / "models" / "sdxl",  # unused by downloader (kept for compatibility)
        hf_token=os.environ.get("MANGA_AI_HF_TOKEN"),
    )
    print("Qwen model is present (downloaded if needed).")
except Exception as e:
    raise RuntimeError(
        "Qwen model download/setup failed. "
        "If Qwen is gated, accept the license on HuggingFace and set MANGA_AI_HF_TOKEN. "
        f"Original error: {e}"
    )


In [ ]:
import sys
import subprocess
from pathlib import Path

req_path = Path(ROOT) / "requirements.txt"
print("Installing from:", req_path)

if not req_path.exists():
    raise FileNotFoundError(f"requirements.txt not found at {req_path}")

# Install into the *current Jupyter kernel* environment
subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", str(req_path)])
print("Note: you may need to restart the kernel after installs.")


In [ ]:
# Quick sanity checks (doesn't load the full ML pipelines)
assert (QWEN_DIR / "config.json").exists(), "Qwen config.json not found"

print("Sanity checks passed.")


In [ ]:
# (Deprecated) SDXL sanity checks removed because Animagine/SDXL was removed from the Qwen-only workflow.
# Keep this cell as a no-op so running top-to-bottom doesn't fail.
print("Skipping SDXL sanity checks (Animagine removed).")


## Launch Web UI + Cloudflare Public URL

Running the next cell will:
- Start Gradio locally on port `7860`
- Start a Cloudflare tunnel and print a public `https://...trycloudflare.com` URL


In [ ]:
from pathlib import Path

from scripts.storyboard import generate_storyboard, save_storyboard

print("Using Qwen model at:", QWEN_DIR)

basic_prompt = """
A shy high-school artist discovers a mysterious sketchbook that makes drawings come to life.
First page: she finds the book after school in an empty art classroom during golden hour.
Focus on emotions, small details, and a hint of supernatural mystery.
"""

sb = generate_storyboard(
    story_prompt=basic_prompt,
    model_dir=QWEN_DIR,
    pages=1,
    max_new_tokens=1600,
    temperature=0.3,
    top_p=0.9,
)

out_path = Path(ROOT) / "outputs" / "storyboard_qwen.json"
save_storyboard(sb, out_path)
print(f"Saved storyboard to: {out_path}")

print("\n=== STORYBOARD SUMMARY ===")
print(f"Title: {sb.title}")
print(f"Logline: {sb.logline}")
print(f"Style notes: {sb.style_notes}")
print(f"\nPages: {len(sb.pages)}")

for page in sb.pages:
    print(f"\n--- Page {page.page_index} ---")
    for panel in page.panels:
        x, y, w, h = panel.bbox_norm
        print(f"\nPanel {panel.panel_id}")
        print(f"  BBox (norm): x={x:.3f}, y={y:.3f}, w={w:.3f}, h={h:.3f}")
        print(f"  Camera: {panel.camera_angle}, shot: {panel.shot}")
        print(f"  Scene: {panel.scene}")
        print(f"  Visual focus: {panel.visual_focus}")
        if panel.characters:
            print(f"  Characters: {', '.join(panel.characters)}")
        if panel.dialogue:
            print("  Dialogue:")
            for line in panel.dialogue:
                tone_str = f" ({line.tone})" if line.tone else ""
                print(f"    [{line.bubble_style}] {line.speaker}{tone_str}: {line.text}")
        print(f"  Prompt: {panel.sdxl_prompt[:180]}...")
